In [15]:

import requests
import json 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os 

from datetime import datetime
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


!pip install scrapy

from urllib.parse import parse_qs, urlparse

import logging
import scrapy
from scrapy.crawler import CrawlerProcess

pio.renderers.default = "colab"
     
    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
villes = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [20]:
position_villes = []

for i in villes:
    response = requests.get(f"https://nominatim.openstreetmap.org/search?q={i}&format=json")
    data = response.json()
    if data:
        latitude = float(data[0]['lat'])
        longitude = float(data[0]['lon'])
        position_villes.append([latitude, longitude])

In [25]:
# Création d'une liste d'ID pour chaque ville dans la liste
ID = [i+1 for i in range(len(villes))]

# Création d'un dataframe contenant les informations de ville, de géolocalisation et d'ID
data_recup = {
    'ville': villes,
    'long': [coord[1] for coord in position_villes],
    'lat': [coord[0] for coord in position_villes],
    'ID': ID
}
nomidata = pd.DataFrame(data_recup)

# Affichage du dataframe
nomidata.head()

,ville,long,lat,ID
0,Mont Saint Michel,-1.511460,48.635954,1
1,St Malo,-2.026041,48.649518,2
2,Bayeux,-0.702474,49.276462,3
3,Le Havre,0.107973,49.493898,4
4,Rouen,1.093966,49.440459,5


In [26]:
nomidata.shape

(35, 4)

In [41]:
from datetime import datetime

weather_data = []

# Boucle sur chaque ville pour récupérer les données météorologiques
for index, ville in enumerate(villes):
    latitude, longitude = position_villes[index]

    # Récupération des données météorologiques avec l'API OpenWeatherMap
    response = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={latitude}&lon={longitude}&units=metric&lang=fr&exclude=minutely,hourly&appid=059029898705fcb71d83dad4cc961624&format=json")
    data = response.json()

    # Extraction des données météorologiques pour chaque jour
    for day_index in range(1, 8):
        date_scrap = data['daily'][day_index]['dt']
        date = datetime.fromtimestamp(date_scrap).strftime("%d/%m/%y")
        weather_description = data['daily'][day_index]['weather'][0]['description']
        temperature_max = data['daily'][day_index]['temp']['max']
        temperature_min = data['daily'][day_index]['temp']['min']
        temperature_day = data['daily'][day_index]['temp']['day']

        # Ajout des données météorologiques à la liste
        weather_data.append([latitude, longitude, ville, index+1, date, weather_description, temperature_max, temperature_min, temperature_day])

# Création d'un dataframe à partir de la liste de données météorologiques
weather_df = pd.DataFrame(weather_data, columns=['lat', 'long', 'ville', 'ID', 'date', 'weather_description', 'temperature_max', 'temperature_min', 'temperature_day'])

# Affichage du dataframe
display(weather_df)


,lat,long,ville,ID,date,weather_description,temperature_max,temperature_min,temperature_day
0,48.635954,-1.511460,Mont Saint Michel,1,13/06/23,légère pluie,23.72,13.66,23.41
1,48.635954,-1.511460,Mont Saint Michel,1,14/06/23,ciel dégagé,24.97,15.10,24.93
2,48.635954,-1.511460,Mont Saint Michel,1,15/06/23,nuageux,24.94,14.42,24.71
3,48.635954,-1.511460,Mont Saint Michel,1,16/06/23,ciel dégagé,25.01,13.17,25.01
4,48.635954,-1.511460,Mont Saint Michel,1,17/06/23,couvert,29.36,15.45,27.84
...,...,...,...,...,...,...,...,...,...
240,46.159113,-1.152043,La Rochelle,35,15/06/23,nuageux,22.49,17.97,22.49
241,46.159113,-1.152043,La Rochelle,35,16/06/23,partiellement nuageux,22.76,17.47,22.58
242,46.159113,-1.152043,La Rochelle,35,17/06/23,couvert,25.89,17.94,24.78
243,46.159113,-1.152043,La Rochelle,35,18/06/23,pluie modérée,24.59,18.31,24.59


In [42]:
weather_df.to_csv('weather_df')


In [43]:
import folium

# Création de la carte
m = folium.Map(location=[47, 2], zoom_start=4)

# Ajout des marqueurs pour chaque ville
for index, row in weather_df.iterrows():
    ville = row['ville']
    latitude = row['lat']
    longitude = row['long']
    temperature_jour = row['temperature_day']
    
    # Création du marqueur
    marker = folium.Marker(
        location=[latitude, longitude],
        popup=f"Ville: {ville}<br>Température jour: {temperature_jour}",
        icon=folium.Icon(color='blue', icon='cloud')
    )
    
    # Ajout du marqueur à la carte
    marker.add_to(m)

# Affichage de la carte
m